In [ ]:
import sys
import numpy as np
import xarray as xr
import pyproj
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cmocean as cmo
import cartopy.crs as ccrs

sys.path.append("..")

from plotfunctions import *
import warnings
warnings.filterwarnings("ignore")

np.seterr(all='ignore')
%matplotlib notebook
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
mpl.rcParams['figure.subplot.bottom'] = .01
mpl.rcParams['figure.subplot.top']    = .94
mpl.rcParams['figure.subplot.left']   = .01
mpl.rcParams['figure.subplot.right']  = .99
mpl.rcParams['figure.subplot.wspace'] = .05
mpl.rcParams['figure.subplot.hspace'] = .15

In [ ]:
"""Boundaries MITgcm"""
y1,y2 = -75.45,-74.05
x1,x2 = 245.3,251

"""Time slice MITgcm"""
timep_w = slice("2006-1-1","2012-1-1")
timep_c = slice("2013-1-1","2017-1-1")

"""Locations for text"""
plon,plat = 250.5,-74.4

"""Parameters Layer"""
date = '2021_02_25/'
forc_c = 'tanh2_Tdeep0.7_ztcl-700'
forc_w = 'tanh2_Tdeep0.7_ztcl-400'
spinup = 180

"""Literature value observations"""
mobs = '5.5-7'

n_w = 2
n_c = 5

In [ ]:
def printvals(ax,melt):
    ax.text(plon,plat,f'{np.nanmean(melt):.1f}',transform=ccrs.PlateCarree(),c='w',ha='center')
    

In [ ]:
proj = ccrs.SouthPolarStereo(true_scale_latitude=-75,central_longitude=245-360)
axex = [245.3,250.8,-75.3,-74.15]

def createax(n,title):
    ax = fig.add_subplot(3,3,n,projection=proj)
    makebackground(ax)
    ax.set_extent(axex,crs=ccrs.PlateCarree())
    ax.set_title(title,loc='left')
    return ax

def weightmelt(melt_w,melt_c):
    return (n_w*melt_w+n_c*melt_c)/(n_w+n_c)

In [ ]:
fig = plt.figure(figsize=(8,7))

"""Geometry"""
ax = createax(1,'a) Geometry')
ds = xr.open_dataset('../../data/BedMachineAntarctica_2020-07-15_v02.nc')
ds = ds.isel(x=slice(3000,4000),y=slice(6500,9000))
ds = add_lonlat(ds)
draft = xr.where(ds.mask==3,(ds.surface-ds.thickness).astype('float64'),np.nan)
IM = plt.pcolormesh(ds.lon,ds.lat,draft,vmin=-1000,vmax=0,cmap=plt.get_cmap('cmo.rain_r'),transform=ccrs.PlateCarree(),shading='auto')

"""Colorbar"""
ax3 = fig.add_subplot(333)
ax3.set_visible(False)
axins = inset_axes(ax3,width="5%",height="100%",loc='lower left',bbox_to_anchor=(.1,0, 1, 1),bbox_transform=ax3.transAxes,borderpad=0)
cbar = plt.colorbar(IM, cax=axins,extend='min')
cbar.set_label('Draft depth [m]', labelpad=0)

"""Observations"""
ax = createax(2,'b) Observations')
ds = xr.open_dataset('../../data/gourmelen/CrossonDotson.nc')
ds = add_lonlat(ds)
melt = ds.Band1
ax.text(plon,plat,mobs,transform=ccrs.PlateCarree(),c='w',ha='center')
IM = plotmelt(ax,ds.lon,ds.lat,melt)

"""Colorbar"""
axins = inset_axes(ax3,width="5%",height="100%",loc='lower left',bbox_to_anchor=(.6,0, 1, 1),bbox_transform=ax3.transAxes,borderpad=0)
cbar = plt.colorbar(IM, cax=axins,extend='both')
cbar.set_ticks([1,10,100])
cbar.set_ticklabels([1,10,100])
cbar.set_label('Melt rate [m/yr]', labelpad=0)

"""M+"""
ax = createax(4,'c) M+')
#Plot melt here

"""PICO"""
ax = createax(5,'d) PICO')
#Plot melt here

"""Plume"""
ax = createax(6,'e) Plume')
#Plot melt here

"""PICOP"""
ax = createax(7,'f) PICOP')
#Plot melt here

"""Layer"""
ax = createax(8,'g) Layer')
ds = xr.open_dataset(f'../../results/Layer/{date}CrossDots_{forc_w}_{spinup:.3f}.nc')
ds = add_lonlat(ds)
melt_w = np.where(ds.mask==3,ds.melt,np.nan)
ds = xr.open_dataset(f'../../results/Layer/{date}CrossDots_{forc_c}_{spinup:.3f}.nc')
ds = add_lonlat(ds)
melt_c = np.where(ds.mask==3,ds.melt,np.nan)
melt = weightmelt(melt_w,melt_c)
melt = np.where(melt<1,1,melt)
printvals(ax,melt)
plotmelt(ax,ds.lon,ds.lat,melt)

"""MITgcm"""
ax = createax(9,'h) MITgcm')
ds = xr.open_dataset('../../data/paulholland/melt.nc')
ds = ds.sel(LONGITUDE=slice(x1,x2),LATITUDE=slice(y1,y2),TIME=timep_w)
ds = ds.mean(dim='TIME')
melt_w = xr.where(ds.melt==0,np.nan,ds.melt)
ds = xr.open_dataset('../../data/paulholland/melt.nc')
ds = ds.sel(LONGITUDE=slice(x1,x2),LATITUDE=slice(y1,y2),TIME=timep_c)
ds = ds.mean(dim='TIME')    
lon = ds.LONGITUDE
lat = ds.LATITUDE-.05
melt_c = xr.where(ds.melt==0,np.nan,ds.melt)
melt = weightmelt(melt_w,melt_c)
printvals(ax,melt)
plotmelt(ax,lon,lat,melt)

"""Save figure"""
plt.savefig(f"../../figures/Validation_CD.png",dpi=300)
plt.show()